For this lab, we will be using the `marketing_customer_analysis.csv` file that you can find in the `files_for_lab` folder. Check out the `files_for_lab/about.md` to get more information if you are using the Online Excel.

**Note**: For the next labs we will be using the same data file. Please save the code, so that you can re-use it later in the labs following this lab.

### Instructions

#### 1. Show the dataframe shape.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('files_for_lab/csv_files/marketing_customer_analysis.csv')
print(df.shape)

#### 2. Standardize header names.

In [ ]:
pd.set_option('max_columns', None)

# Check header names & data
print(df.columns)

# Check what Unnamed: 0 column is
print(df['Unnamed: 0'])

# As the Unnamed: 0 column is identical to the index column, we can drop it
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# Review the rest of the data
print(df.head())

# No other duplicates seen, so will proceed with removing the typo in one of the column names and changing the font to lowercase:
df.rename(columns = {'EmploymentStatus': 'Employment Status'}, inplace = True)
df.columns = df.columns.str.lower()
print(df.columns)

#### 3. Which columns are numerical?

- Customer Lifetime Value
- Income
- Monthly Premium Auto
- Months Since Last Claim
- Months Since Policy Inception
- Number of Open Complaints
- Number of Policies
- Total Claim Amount

#### 4. Which columns are categorical?

All the remaining columns.

#### 5. Check and deal with `NaN` values.

First off, we'll look at the amount of null values within the numerical columns.

In [ ]:
null_values = df.isna().sum()

print(null_values)

Now, we'll compare this to the size of the data frame:

In [ ]:
null_values_percentage = null_values * 100 / df.shape[0]
print(null_values_percentage)

Given that the vehicle type column has 1/2 missing values, we can drop that column. 

In [ ]:
df.drop('vehicle type', axis = 1, inplace = True)

# Check that the drop was successful
print(df.shape)
print(df.head())

Now let's check how many rows have null values:

In [ ]:
null_dataframe = df[(df['state'].isna()) | (df['response'].isna()) 
| (df['months since last claim'].isna()) | (df['number of open complaints'].isna())
| (df['vehicle class'].isna()) | (df['vehicle size'].isna())]

print(null_dataframe.shape)
print(null_dataframe.head())

Given the rows with at least one NaN value make up for ~16% of the dataframe, we can only remove very few of these rows. Let's check how many rows have NaN values in the categorical columns _state_ and _response_:

In [ ]:
null_dataframe = df[(df['state'].isna()) | (df['response'].isna())]
print(null_dataframe.shape)

Should we replace the NaN values with the modes of these columns?

In [ ]:
states_count = df['state'].value_counts()
response_count = df['response'].value_counts()

print(states_count)
print(response_count)

We could choose California as the default state, however Oregon & Arizona are relatively close contenders. Therefore, we'll drop the rows, which will automatically remove the null values in the _response_ column:

In [ ]:
df = df[(df['state'].isna() == False) | (df['response'].isna() == False)]

#Check that the rows were droped
print(df.shape)

We're going to follow the same thinking for the _vehicle class_ and _vehicle size_ categorical columns:

In [ ]:
vehicle_class_count = df['vehicle class'].value_counts()
vehicle_size_count = df['vehicle size'].value_counts()

print(vehicle_class_count)
print(vehicle_size_count)

In this case, it is more obvious what the mode for each columns is, so we'll replace the null values with that:

In [ ]:
vehicle_class_mode = df['vehicle class'].mode()[0]
vehicle_size_mode = df['vehicle size'].mode()[0]

df['vehicle class'] = df['vehicle class'].fillna(vehicle_class_mode)
df['vehicle size'] = df['vehicle size'].fillna(vehicle_size_mode)

# Check if there are still NaN values in the columns
print(df['vehicle class'].isna().sum())
print(df['vehicle size'].isna().sum())

For the rest of the columns, we can replace the null values with the mean value:

In [ ]:
# Check that the columns have float/int data types:
# print(df.dtypes)

# Since the 'months since last claim' and 'number of open complaints' columns are already float64 types,
# we can calculate their mean:
mean_months = np.mean(df['months since last claim'])
mean_open_complaints = np.mean(df['number of open complaints'])

df['months since last claim'] = df['months since last claim'].fillna(mean_months)
df['number of open complaints'] = df['number of open complaints'].fillna(mean_open_complaints)

# Check if there are still NaN values in the columns
print(df['months since last claim'].isna().sum())
print(df['number of open complaints'].isna().sum())

#### 6. Datetime format - Extract the months from the dataset and store in a separate column. Then filter the data to show only the information for the first quarter , ie. January, February and March. Hint: If data from March does not exist, consider only January and February.

In [ ]:
# Check the data type in the column
print(df['effective to date'].dtypes)

# Convert to datetime format
df['effective to date'] = pd.to_datetime(df['effective to date'], errors='coerce')

# Check the data type again
print(df['effective to date'].dtypes)
print(df['effective to date'].head())

# Extract the months from the dataset
df['months'] = df.apply(lambda x: x['effective to date'].month, axis = 1)

# Check the dataframe again
df['months'].head()

# Filter the data to show only first quarter info
df = df[(df['months'] > 0) & (df['months'] < 4)]

# Check that the data was filtered
print(df['months'].unique())

#### Put all the previously mentioned data transformations into a function.

In [ ]:
# Re-reading the data to test the function
df_new = pd.read_csv('files_for_lab/csv_files/marketing_customer_analysis.csv')

# will keep the most general items
def big_clean(df):

    # Standardize column names
    df.columns = df.columns.str.lower()

    # Clean NaN values & extra index column. Will replace all NaN by the mode in this case
    for column in df.columns:
        if (df[column].isna().sum() / df.shape[0]) > 0.5:
            df.drop(column, axis = 1, inplace = True)
        elif (df[column].isna().sum() > 0):
            if df[column].dtypes == object:
                df[column] = df[column].fillna(df[column].mode()[0])
            else:
                df[column] = df[column].fillna(np.mean(df[column]))
    
    return df.head(), df.isna().sum().sum()

# Checking if the formula works
big_clean(df_new)